In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/econ.csv', encoding='ISO-8859-1')
df.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               8000 non-null   int64  
 1   _golden                8000 non-null   bool   
 2   _unit_state            8000 non-null   object 
 3   _trusted_judgments     8000 non-null   int64  
 4   _last_judgment_at      8000 non-null   object 
 5   positivity             1420 non-null   float64
 6   positivity:confidence  3775 non-null   float64
 7   relevance              8000 non-null   object 
 8   relevance:confidence   8000 non-null   float64
 9   articleid              8000 non-null   object 
 10  date                   8000 non-null   object 
 11  headline               8000 non-null   object 
 12  positivity_gold        0 non-null      float64
 13  relevance_gold         0 non-null      float64
 14  text                   8000 non-null   object 
dtypes: b

In [ ]:
df['relevance'].value_counts()

no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [ ]:
df_ns = df[df.relevance != 'not sure']
df_ns['relevance'] = df_ns.relevance.map({'yes':1,'no':0})
df_cleaned = df_ns[['text','relevance']]
df_cleaned

<ipython-input-112-40cb4bf48a8e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ns['relevance'] = df_ns.relevance.map({'yes':1,'no':0})


,text,relevance
0,NEW YORK -- Yields on most certificates of dep...,1
1,The Wall Street Journal Online</br></br>The Mo...,0
2,WASHINGTON -- In an effort to achieve banking ...,0
3,The statistics on the enormous costs of employ...,0
4,NEW YORK -- Indecision marked the dollar's ton...,1
...,...,...
7995,Secretary of Commerce Charles W. Sawyer said y...,1
7996,"U.S. stocks inched up last week, overcoming co...",0
7997,Ben S. Bernanke cleared a key hurdle Thursday ...,0
7998,The White House's push to contract out many fe...,0


In [ ]:
sw = _stop_words.ENGLISH_STOP_WORDS
def clean_text(headline):
  headline = re.sub(r'</br>',' ', headline)
  headline = ' '.join([i for i in headline.split() if i not in sw])
  headline = ''.join([char for char in headline if char not in string.punctuation and not char.isdigit()])
  return headline
clean_text("omg </br> you  suck/ . ./3./3")

'omg suck  '

In [ ]:
X = df_cleaned[['text']]
y = df_cleaned.relevance
X['text'].apply(clean_text)
X = X.text
X.shape

(7991,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5993,) (5993,)
(1998,) (1998,)


In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

X_train_features = vectorizer.transform(X_train)
X_test_features = vectorizer.transform(X_test)

print(X_train_features.shape)
print(X_test_features.shape)

(5993, 39728)
(1998, 39728)


In [ ]:
model = MultinomialNB()
model.fit(X_train_features, y_train)
y_results = model.predict(X_test_features)
y_results

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_results))
print("Precision:", precision_score(y_test, y_results, average='weighted'))
print("Recall:", recall_score(y_test, y_results, average='weighted'))
print("F1 score:", f1_score(y_test, y_results, average='weighted'))
print(classification_report(y_test, y_results))

Accuracy: 0.7527527527527528
Precision: 0.7885322280318793
Recall: 0.7527527527527528
F1 score: 0.7676191452273584
              precision    recall  f1-score   support

           0       0.88      0.81      0.84      1656
           1       0.34      0.47      0.39       342

    accuracy                           0.75      1998
   macro avg       0.61      0.64      0.62      1998
weighted avg       0.79      0.75      0.77      1998



In [ ]:
test = np.array(["rising interest rates. small denominations"])
test.shape

test_features = vectorizer.transform(test)
print(test_features)
print(model.predict(test_features))

  (0, 11053)	1
  (0, 18939)	1
  (0, 28123)	1
  (0, 29625)	1
  (0, 31862)	1
[0]
